## Libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

import geocoder
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Importing the Dataset

In [2]:
url = 'https://en.wikipedia.org/wiki/Category:Suburbs_of_Melbourne'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.findAll('div', {'class': "mw-category-group"})

In [3]:
suburbs = []
for tag in soup.find_all("li"):
    if(', Victoria' in tag.text):
        text = tag.text
        i = 0
        while(not text[i].isalpha()):
            i = i + 1        
        
        suburbs.append(tag.text[i:tag.text.index(", Victoria")+10])

len(suburbs)

212

In [7]:
# create a new DataFrame from the list
melb_sub = pd.DataFrame({"Suburbs": suburbs})

melb_sub.head()

,Suburbs
0,"Broadmeadows, Victoria"
1,"Dandenong, Victoria"
2,"East Melbourne, Victoria"
3,"Elsternwick, Victoria"
4,"Essendon, Victoria"


## Retrieving Geographical Coordinates

In [6]:
# code for getting the latitude and longitude
def get_lati_long(suburb):
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Melbourne, Victoria'.format(suburb))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
suburb_names = melb_sub['Suburbs'].tolist()

In [11]:
lat_lang = [get_lati_long(suburb) for suburb in suburb_names]

In [12]:
df_coords = pd.DataFrame(lat_lang, columns=['Latitude', 'Longitude'])
melb_sub['Latitude'] = df_coords['Latitude']
melb_sub['Longitude'] = df_coords['Longitude']

In [13]:
melb_sub.head()

,Suburbs,Latitude,Longitude
0,"Broadmeadows, Victoria",-37.686040,144.926100
1,"Dandenong, Victoria",-37.959885,145.208850
2,"East Melbourne, Victoria",-37.810043,144.985531
3,"Elsternwick, Victoria",-37.887322,145.009896
4,"Essendon, Victoria",-37.751530,144.909510


In [14]:
melb_sub.to_csv('melb.csv',index=False)

## Getting Location Data of Melbourne

In [15]:
melb_data = pd.read_csv('melb.csv')
melb_data.head()

,Suburbs,Latitude,Longitude
0,"Broadmeadows, Victoria",-37.686040,144.926100
1,"Dandenong, Victoria",-37.959885,145.208850
2,"East Melbourne, Victoria",-37.810043,144.985531
3,"Elsternwick, Victoria",-37.887322,145.009896
4,"Essendon, Victoria",-37.751530,144.909510


In [16]:
address = 'Melbourne, Victoria, Australia'

geolocator = Nominatim(user_agent="melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Melbourne, Australia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Melbourne, Australia are -37.8142176, 144.9631608.


In [18]:
neighborhood_latitude = melb_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = melb_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = melb_data.loc[0, 'Suburbs'] # neighborhood name

In [21]:
# create map of Manhattan using latitude and longitude values
map_melbourne = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, label in zip(melb_data['Latitude'], melb_data['Longitude'], melb_data['Suburbs']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_melbourne)  
    
map_melbourne

## Using Foursquare to get the Venues in various Suburbs in Melbourne

In [22]:
CLIENT_ID = '2H5JYVFDJK2UJZ42OQZ0HBSGQGF0IBBIAZSZGLEN2MXERZ1W' # your Foursquare ID
CLIENT_SECRET = 'BWSLIRMBGF35FIZAWIA1UXUT3PWX415CZCIKQQ30XEQEO2T0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2H5JYVFDJK2UJZ42OQZ0HBSGQGF0IBBIAZSZGLEN2MXERZ1W
CLIENT_SECRET:BWSLIRMBGF35FIZAWIA1UXUT3PWX415CZCIKQQ30XEQEO2T0


In [23]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2H5JYVFDJK2UJZ42OQZ0HBSGQGF0IBBIAZSZGLEN2MXERZ1W&client_secret=BWSLIRMBGF35FIZAWIA1UXUT3PWX415CZCIKQQ30XEQEO2T0&v=20180605&ll=-37.686039999999934,144.92610000000005&radius=500&limit=100'

In [24]:
results = requests.get(url).json()

## Get all the Nearby Venues in Melbourne

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburbs', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# type your answer here
melb_venues = getNearbyVenues(names=melb_data['Suburbs'], latitudes=melb_data['Latitude'], longitudes=melb_data['Longitude'])

Broadmeadows, Victoria
Dandenong, Victoria
East Melbourne, Victoria
Elsternwick, Victoria
Essendon, Victoria
Fitzroy, Victoria
Flemington, Victoria
Frankston, Victoria
Glen Waverley, Victoria
Heidelberg, Victoria
Maribyrnong, Victoria
St Kilda East, Victoria
St Kilda, Victoria
Sunshine, Victoria
Werribee, Victoria
Williamstown, Victoria
Abbotsford, Victoria
Aberfeldie, Victoria
Aintree, Victoria
Airport West, Victoria
Albanvale, Victoria
Albert Park, Victoria
Albion, Victoria
Alphington, Victoria
Altona Meadows, Victoria
Altona North, Victoria
Altona, Victoria
Ardeer, Victoria
Armadale, Victoria
Ascot Vale, Victoria
Ashburton, Victoria
Ashwood, Victoria
Aspendale Gardens, Victoria
Aspendale, Victoria
Attwood, Victoria
Auburn, Victoria
Aurora, Victoria
Avondale Heights, Victoria
Balaclava, Victoria
Balwyn, Victoria
Bayswater North, Victoria
Bayswater, Victoria
Beaconsfield, Victoria
Beaumaris, Victoria
Belgrave Heights, Victoria
Belgrave South, Victoria
Belgrave, Victoria
Bellfield, Vic